In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/oddrationale/mnist-in-csv")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: IanD24
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/oddrationale/mnist-in-csv


100%|██████████| 15.2M/15.2M [00:00<00:00, 1.05GB/s]

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available()else 'cpu'

In [3]:
# Import MNIST datasets from Kaggle
train_df = pd.read_csv("/content/mnist-in-csv/mnist_train.csv")
test_df = pd.read_csv("/content/mnist-in-csv/mnist_test.csv")


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Clearning and Normalizing data by deviding by largest values in columns (255 in this case)
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

test_labels = np.array(test_df['label'])
train_labels = np.array(train_df['label'])

train_df.drop(columns='label', inplace=True)
test_df.drop(columns='label', inplace=True)

train_df = np.array(train_df) / 255
test_df = np.array(test_df) / 255

In [5]:
# Creating Dataset class to allow np arrarys to be converted to tensors for use with pytorch module
class dataset(Dataset):
  def __init__(self, X, Y):
    self.X = torch.tensor(X, dtype = torch.float).to(device)
    self.Y = torch.tensor(Y, dtype = torch.long).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.Y[index]

In [6]:
# Assigning data to variables for training, validation, and testing data
training_data = dataset(train_df[ : int(0.8 * len(train_df)), :], train_labels[ :int(0.8 * len(train_labels))])
validation_data = dataset(train_df[ int(0.8 * len(train_df)) : , :], train_labels[int(0.8 * len(train_labels)) : ])
testing_data = dataset(test_df, test_labels)

In [7]:
# Creating dataloader iterables for batching data in training
train_dataloader = DataLoader(training_data, batch_size = 16, shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size = 16, shuffle = True)
testing_dataloader = DataLoader(testing_data, batch_size = 16, shuffle = True)

In [8]:
# Setting variables for layer sizes and loss function
LAYER_1 = 140
LAYER_2 = 60
LAYER_3 = 50

OUTPUT_NEURONS = 10

criterion = nn.CrossEntropyLoss()

In [9]:
# Buildingl class for my neural network including the types of layers and activation functions
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()

    self.input_layer = nn.Linear(train_df.shape[1], LAYER_1)
    self.hidden1 = nn.Linear(LAYER_1, LAYER_2)
    self.hidden2 = nn.Linear(LAYER_2, LAYER_3)
    self.hidden3 = nn.Linear(LAYER_3, OUTPUT_NEURONS)

    self.relu = nn.ReLU()
    self.drop = nn.Dropout(p=0.1)

  def forward(self, x):
    x = self.relu(self.input_layer(x))
    x = self.relu(self.hidden1(x))
    x = self.hidden2(x)
    return x


model = MyModel().to(device)
optimizer = Adam(model.parameters(), lr = 1e-3)

epochs = 3

for epoch in range(epochs):
  total_acc_train = 0
  total_loss_train = 0
  total_acc_val = 0
  total_loss_val = 0

  # Training
  for data in train_dataloader:
    inputs, labels = data
    prediction = model(inputs)
    batch_loss = criterion(prediction, labels)

    total_loss_train += batch_loss.item()

    predicted_classes = prediction.argmax(dim=1)
    acc = ((predicted_classes == labels)).sum().item()
    total_acc_train += acc

    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  # Validation
  with torch.no_grad():
    for data in validation_dataloader:
      inputs, labels = data
      prediction = model(inputs)
      batch_loss = criterion(prediction, labels)

      total_loss_val += batch_loss.item()

      predicted_classes = prediction.argmax(dim=1)
      acc = ((predicted_classes == labels)).sum().item()
      total_acc_val += acc

train_accuracy = total_acc_train / len(training_data)
validation_accuracy = total_acc_val / len(validation_data)

print(f'Training Accuracy: {train_accuracy * 100}%')
print(f'Validation Accuracy: {validation_accuracy * 100}%')


Training Accuracy: 97.40416666666667%
Validation Accuracy: 97.1%
